# Tutorial: writing serverless package from scratch

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

---

There are couple of simple rules that you need to follow in order to write software packages with serverless.

- Hide all unnecessary details from user to improve user experience
  - hide ray library calls
  - user should not be aware of ray if he do not need to
- request as little resources as possible, it is better to have 10 function calls which require 1 cpu, than 1 function call which requires 10 cpus
- in general follow patterns and avoid antipatterns by ray.io https://docs.ray.io/en/releases-1.9.0/ray-design-patterns/index.html

In [2]:
from typing import Optional, List, Dict, Any

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator, EstimatorResult
from qiskit.circuit.random import random_circuit
from qiskit.opflow import X, I

from quantum_serverless import QuantumServerless, put, get, run_qiskit_remote

In [3]:
# classical funciton that will be executed as remote task
@run_qiskit_remote()
def add_prefix(circuit: QuantumCircuit, prefix_depth) -> QuantumCircuit:
    return random_circuit(circuit.num_qubits, prefix_depth).compose(circuit)

# quantum related compute that should be executed as closer to quantum as possible
# using resource QPU
@run_qiskit_remote()
def measure_exp_val(circuits: List[QuantumCircuit], observable) -> EstimatorResult:
    with Estimator(circuits, observable) as estimator:
        return estimator(range(len(circuits)), [0] * len(circuits))

# this class is hiding all implementation details and ray annotation from user
class YourClass:
    def __init__(self, observable, prefix_depth: int = 3):
        self.prefix_depth = prefix_depth
        self.observable = observable
    
    def routine(self, circuits: List[QuantumCircuit]) -> EstimatorResult:
        new_circuits = get([
            add_prefix(circuit, self.prefix_depth)
            for circuit in circuits
        ])
        return get(measure_exp_val(new_circuits, self.observable))

# user only interact with QuantumServerless oobject to set compute resource and 
# high level YourClass class
circuits = [random_circuit(3, 3) for _ in range(3)]
observable = SparsePauliOp.from_list([("III", 1), ("IIZ", 2), ("IXI", 3)])

serverless = QuantumServerless()

with serverless.context():
    res = YourClass(observable=observable, prefix_depth=3).routine(circuits)
    print(f"Cluster result {res}")

Cluster result EstimatorResult(values=array([2.15866755, 1.37974414, 3.        ]), metadata=[{}, {}, {}])
